In [1]:
from ase.build import bulk

import numpy as np

import pyiron_workflow as pwf
from pyiron_workflow import Workflow
from pyiron_workflow_atomistics.bulk import eos_volume_scan, get_equil_lat_param, get_bulk_structure
from pyiron_workflow_atomistics.calculator import calculate_structure_node
from pyiron_workflow_atomistics.gb.gb_code.constructor import construct_GB_from_GBCode
from pyiron_workflow_atomistics.calculator import ase_calculate_structure_node_interface

from tensorpotential.calculator import TPCalculator

from pyiron_workflow_lammps.lammps import lammps_job, LammpsInput
from pyiron_workflow_lammps.calculator import lammps_calculator_node
import os
%load_ext autoreload
%autoreload 2

2025-06-14 01:54:41.315061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-14 01:54:41.372305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-14 01:54:41.372396: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 01:54:41.413177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-14 01:54:43.955375: W tensorflow/compiler/tf

In [5]:
lmp_input = LammpsInput(
    read_data_file="lammps.data",
    pair_coeff=(
        "*", "*",
        "/root/github_dev/test_workflow_nodes/2025_04_29_FeGB_Segregation_Workflows/final_model",
        "C Fe"
    ),
    dump_every=10,
    dump_filename="dump.out",
    thermo_every=10,
    min_style="cg",
    minimize_params=(0, 0.01, 1000000, 1000000),
)

working_directory = os.path.join(os.getcwd(), "test_lammps_calculator/pyiron_lammps_test")

structure = bulk("Fe", cubic=True) * (3, 3, 3)
structure.rattle(0.3)
structure[1].symbol = "C"
potential_elements = ["C", "Fe"]

lammps_calculator_input = {
    "working_directory": working_directory,
    "lmp_input": lmp_input,
    "potential_elements": potential_elements,
    "input_filename": "in.lmp",
    "command": "/root/github_dev/lammps/build/lmp -in in.lmp -log minimize.log",
    "lammps_log_filepath": "minimize.log",
    "lammps_log_convergence_printout": "Total wall time:",
}

wf = Workflow("test_lammps_calculator", delete_existing_savefiles=True)
# wf.lmp_run = lammps_job(structure = structure,
#                         lmp_input = lmp_input,
#                         potential_elements = potential_elements,
#                         working_directory = working_directory,
#                         input_filename = "in.lmp",
#                         command = "/root/github_dev/lammps/build/lmp -in in.lmp -log minimize.log")
wf.lmp_run = calculate_structure_node(structure = structure,
                                    calc_structure_fn = lammps_calculator_node,
                                    calc_structure_fn_kwargs = lammps_calculator_input)

wf.run()

/root/github_dev/pyiron_workflow_lammps/pyiron_workflow_lammps/generic.py:169: UserWarning: Directory '/root/github_dev/test_workflow_nodes/test_lammps_calculator/pyiron_lammps_test' already exists. Existing files may be overwritten.
  warnings.warn(


/root/github_dev/test_workflow_nodes/test_lammps_calculator/pyiron_lammps_test


{'lmp_run__atoms': Atoms(symbols='FeCFe52', pbc=True, cell=[[8.61, 5.272104470329355e-16, 5.272104470329355e-16], [0.0, 8.61, 5.272104470329355e-16], [0.0, 0.0, 8.61]]),
 'lmp_run__results': {'energy': -442.924301341008,
  'forces': array([[ 2.18897753e-04, -6.12739601e-04,  1.61814179e-04],
         [ 8.38105879e-04, -1.03174454e-04,  8.87339788e-04],
         [-3.43467670e-04, -1.17186308e-04, -1.79690498e-04],
         [-7.23538223e-05, -2.63930801e-04,  1.52736274e-04],
         [-1.28828752e-04, -4.46153738e-04,  6.11878346e-04],
         [-1.04907722e-04,  1.05089935e-03, -5.86026151e-04],
         [-6.14402205e-05, -4.84994202e-04,  3.31359747e-04],
         [ 3.54369266e-04,  2.71515746e-04, -6.65973313e-04],
         [ 6.22975675e-04, -5.94329337e-04,  9.79562563e-05],
         [ 1.03639781e-03, -1.96284848e-04, -1.65861289e-04],
         [ 1.75039475e-04, -3.58746034e-04, -1.26622849e-04],
         [ 3.26563741e-04,  2.01073984e-04, -4.01681336e-04],
         [-3.47965314e-04

In [3]:
wf.lmp_run.outputs.lammps_output["generic"].value["indices"]

AttributeError: Could not find attribute lammps_output on OutputsWithInjection object nor in its channels (['atoms', 'results', 'converged'])

In [15]:
atoms

Atoms(symbols='H215He', pbc=True, cell=[7.34776887, 7.99923754, 60.69771266352846], c_eng=..., momenta=..., calculator=SinglePointCalculator(...))

In [16]:
traj[0]

Atoms(symbols='H215He', pbc=True, cell=[7.34776887, 7.99923754, 60.69771266352846], c_eng=..., momenta=..., calculator=SinglePointCalculator(...))

In [14]:
from ase.io import read

traj = read("/root/github_dev/test_workflow_nodes/gb_seg_lammps/S3_RA110_S112_C_14/dump.out", index=":", format="lammps-dump-text")
# per_frame_arrays = [atoms.get_array('c_pe') for atoms in traj]
# total_pot_energies = [arr.sum() for arr in per_frame_arrays]

total_energies = []
for i, atoms in enumerate(traj):
    # per-atom energy array from your c_eng column
    eng = atoms.get_array('c_eng')        # shape (n_atoms,)

    # total potential energy of the frame
    E_tot = eng.sum()                     # scalar

    # forces and velocities
    forces = atoms.get_forces()           # shape (n_atoms, 3)
    velocities = atoms.get_velocities()   # shape (n_atoms, 3)

    print(f"Frame {i}:  Total energy = {E_tot:.6f}  eV")
    # if you want per-atom info, you already have it in `eng`, `forces`, `velocities`
    total_energies.append(E_tot)

calc = atoms.calc          # or atoms.get_calculator()

# 1) What kind of calculator is it?
print(calc)
# e.g. <SinglePointCalculator: attached to Atoms object>

# 2) What data does it hold?
print(calc.results.keys())
# e.g. dict_keys(['energy', 'free_energy', 'forces', 'stress', ...])

# 3) Access the raw results dict
results = calc.results

# 4) Pull out individual quantities
E_pot      = results.get('energy')           # total potential energy (eV)
F          = results.get('forces')           # (N×3) force array (eV/Å)
# stress     = results.get('stress')           # 6‐component virial stress (Voigt)
free_eng   = results.get('free_energy')      # if your calculator computed it
magmoms    = results.get('magmoms')          # local magnetic moments, if any
charges    = results.get('charges')          # Bader/DFT charges, if any
# …and any other keys you see in results.keys()

# 5) Convenience wrappers on the Atoms object
E_wrap     = atoms.get_potential_energy()    # calls calc.results['energy']
_forces    = atoms.get_forces()              # calls calc.results['forces']
# _stress    = atoms.get_stress()              # calls calc.results['stress']
# (and atoms.get_volume(), atoms.get_kinetic_energy(), etc.)

print(f"Potential energy (raw): {E_pot:.6f} eV  |  wrapper: {E_wrap:.6f} eV")
print("Forces shape:", _forces.shape)
# print("Stress tensor (Voigt):", _stress)

Frame 0:  Total energy = -1755.840388  eV
Frame 1:  Total energy = -1756.617143  eV
Frame 2:  Total energy = -1757.315022  eV
Frame 3:  Total energy = -1757.325429  eV
Frame 4:  Total energy = -1757.327462  eV
Frame 5:  Total energy = -1757.328124  eV
Frame 6:  Total energy = -1757.339050  eV
Frame 7:  Total energy = -1757.369496  eV
Frame 8:  Total energy = -1757.383873  eV
Frame 9:  Total energy = -1757.385076  eV
Frame 10:  Total energy = -1757.385847  eV
Frame 11:  Total energy = -1757.386616  eV
Frame 12:  Total energy = -1757.386806  eV
SinglePointCalculator(energy=0.0, forces=...)
dict_keys(['energy', 'forces'])
Potential energy (raw): 0.000000 eV  |  wrapper: 0.000000 eV
Forces shape: (216, 3)


In [ ]:
from pymatgen.io.lammps.outputs import parse_lammps_dumps, parse_lammps_log

# This will iterate over every snapshot in dump.out
for dump in parse_lammps_dumps("/root/github_dev/test_workflow_nodes/test_lammps_calculator/pyiron_lammps_test/dump.out"):
    print(f"Timestep: {dump.timestep}, Atoms: {dump.natoms}")
    df = dump.data   # pandas DataFrame with your dumped columns
    print(df.head()) # e.g. id, type, x, y, z, fx, fy, fz, vx, vy, vz, c_eng, …
    print(df.type)
    # To get total energy (if you dumped c_eng):
    if "c_eng" in df:
        total_E = df["c_eng"].sum()
        print(f"Total energy (sum of c_eng): {total_E:.6f} eV")


Timestep: 0, Atoms: 54
   id  type       xsu       ysu       zsu        fx        fy        fz  vx  \
0   1     2  0.017307 -0.004818  0.022568  1.056215  0.196476 -0.103923   0   
1   2     1  0.219734  0.158508  0.158509 -4.220903  1.835352 -4.964006   0   
2   3     1  0.055025  0.026740  0.316975 -4.069876 -0.830315  0.257503   0   
3   4     1  0.185571  0.150520  0.483772 -1.932343  1.114126  2.702613   0   
4   5     1  0.008431 -0.066665  0.606565 -4.261521  7.111376  5.641220   0   

   vy  vz     c_eng  
0   0   0 -6.769014  
1   0   0 -7.337997  
2   0   0 -7.542985  
3   0   0 -7.530786  
4   0   0 -5.961185  
0     2
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46

In [42]:
atoms.get_calculator()

/tmp/ipykernel_1537/730559595.py:1: FutureWarning: Please use atoms.calc
  atoms.get_calculator()


In [30]:
traj = read("/root/github_dev/test_workflow_nodes/gb_seg_lammps/S3_RA110_S112_C_14/dump.out", index=":", format="lammps-dump-text")
per_frame_arrays = [atoms.get_array('c_pe') for atoms in traj]
total_pot_energies = [arr.sum() for arr in per_frame_arrays]

total_energies = []
for i, atoms in enumerate(traj):
    # per-atom energy array from your c_eng column
    eng = atoms.get_array('c_eng')        # shape (n_atoms,)

    # total potential energy of the frame
    E_tot = eng.sum()                     # scalar

    # forces and velocities
    forces = atoms.get_forces()           # shape (n_atoms, 3)
    velocities = atoms.get_velocities()   # shape (n_atoms, 3)

    print(f"Frame {i}:  Total energy = {E_tot:.6f}  eV")
    # if you want per-atom info, you already have it in `eng`, `forces`, `velocities`
    total_energies.append(E_tot)

KeyError: 'c_pe'